# Descriptors Wrokgraph Calculation

To create a workgraph and run a calculation, you have to define some inputs as AiiDA data types and build a workgraph. 

First we can import the profile

In [4]:
from aiida import load_profile
load_profile()

Profile<uuid='468a9d4d0812492ba52462f2aca53e23' name='presto'>

First of all we need a structure on which to perform the calculations. It will be a NaCl structure that we define using ASE, or alternatively one can choose one of the structures in the folder `Structures`.

The input structure in aiida-mlip needs to be saved as a StructureData type:

In [5]:
from aiida.orm import StructureData
from ase.build import bulk
from ase.io import read

# structure = StructureData(ase=read("lj-traj.xyz"))
structure = StructureData(ase=bulk("NaCl", "rocksalt", 5.63))

In [6]:
structure.get_ase()

Atoms(symbols='NaCl', pbc=True, cell=[[0.0, 2.815, 2.815], [2.815, 0.0, 2.815], [2.815, 2.815, 0.0]], masses=...)

Then we need to choose a model and architecture to be used for the calculation and save it as ModelData type, a specific data type of this plugin.
In this example we use MACE with a model that we download from this URL: "https://github.com/stfc/janus-core/raw/main/tests/models/mace_mp_small.model", and we save the file in the cache folder (default="~/.cache/mlips/"):


In [7]:
from aiida_mlip.data.model import ModelData

uri = "https://github.com/stfc/janus-core/raw/main/tests/models/mace_mp_small.model"
model = ModelData.from_uri(uri, architecture="mace_mp", cache_dir="mlips")

Another parameter that we need to define as AiiDA type is the code. Assuming the code is saved as `janus` in the `localhost` computer, the code info that are needed can be loaded as follow:

In [8]:
from aiida.orm import load_code

code = load_code("janus@localhost")

The other inputs can be set up as AiiDA Str. There is a default for every input except the structure and code. This is a list of possible inputs:

In [9]:
from aiida.orm import Bool, Str

inputs = {
    "code": load_code("janus@localhost"),
    "model": model,
    "struct": structure,
    "arch": Str(model.architecture),
    "precision": Str("float64"),
    "device": Str("cpu"),
    "metadata": {"options": {"resources": {"num_machines": 1}}},
    "invariants_only": Bool(True),
    "calc_per_atom": Bool(True),
    "calc_per_element": Bool(True),
}

We then load the calculation using `CalculationFactory` and the entrypoint, `"mlip.descriptors"` in this case:

In [10]:
from aiida.plugins import CalculationFactory

DescriptorsCalc = CalculationFactory("mlip.descriptors")

Now we can start building the workgraph. First we use `graph_builder` to get the inputs and run the calculation

In [11]:
from aiida_workgraph import WorkGraph, task
from aiida.engine import run_get_node, CalcJob, WorkChain

@task.graph_builder(outputs=[{"name" : "final_structure", "from": "ctx.final_structure"}])
def calculation(
    calc: CalcJob,
    calc_inputs: dict,
    ):

    wg = WorkGraph("Run Calculation")
    wg.add_input("workgraph.any", "calc_inputs")

    calc_task = wg.add_task(
        calc,
        name="run_calc",
        **calc_inputs
        )

    wg.update_ctx(
        {"final_structure": calc_task.outputs.xyz_output}
    )
    wg.outputs.final_structure = wg.ctx.final_structure

    return wg

Once the task has been created, we can then create a workgraph to pass in the inputs

In [12]:
from aiida_workgraph import WorkGraph

wg = WorkGraph("Descriptors Calculation")
wg.add_input("workgraph.any", "calc_inputs")

wg.add_task(
    calculation, 
    calc = DescriptorsCalc,
    name="descriptors_calc",
    calc_inputs = inputs
    )

wg.outputs.final_structure = wg.tasks.descriptors_calc.outputs.final_structure

Now we can visualise the tasks

In [13]:
wg

NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'Descrip…

If everything has been setup correctly we can then run the workgraph, which runs the calculations with the inputs and get an output

In [14]:
wg.run()

07/22/2025 10:10:47 AM <363187> aiida.broker.rabbitmq: [WARNING] RabbitMQ v3.12.1 is not supported and will cause unexpected problems!
07/22/2025 10:10:47 AM <363187> aiida.broker.rabbitmq: [WARNING] It can cause long-running workflows to crash and jobs to be submitted multiple times.
07/22/2025 10:10:47 AM <363187> aiida.broker.rabbitmq: [WARNING] See https://github.com/aiidateam/aiida-core/wiki/RabbitMQ-version-to-use for details.
07/22/2025 10:10:52 AM <363187> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [7400|WorkGraphEngine|continue_workgraph]: tasks ready to run: descriptors_calc


defining outputnode


07/22/2025 10:10:54 AM <363187> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [7400|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 7401
07/22/2025 10:10:55 AM <363187> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [7401|WorkGraphEngine|continue_workgraph]: tasks ready to run: run_calc
07/22/2025 10:10:57 AM <363187> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [7401|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 7404
07/22/2025 10:11:31 AM <363187> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [7401|WorkGraphEngine|update_task_state]: Task: run_calc, type: CALCJOB, finished.
07/22/2025 10:11:32 AM <363187> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [7401|WorkGraphEngine|continue_workgraph]: tasks ready to run: 
07/22/2025 10:11:32 AM <363187> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [7401|WorkGraphEngine|f

We can check if the graph has given us the correct output flle


In [15]:
output_struct = wg.outputs.final_structure.value

In [16]:
import shutil

with output_struct.open(mode='rb') as source:
    with open('output.xyz', mode='wb') as target:
        shutil.copyfileobj(source, target)

In [18]:
! janus descriptors --struct lj-traj.xyz --arch mace_mp --calc-per-element --calc-per-atom 

/home/mtr46585/aiida-mlip/.venv/lib/python3.12/site-packages/e3nn/o3/_wigner.py:10: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  _Jd, _W3j_flat, _W3j_indices = torch.load(os.path.join(os.path.dirname(__file__), 'constants.pt'))
cuequivariance or cuequivariance_torch is not available. Cuequivariance acceleration will be disabled.
Using Materials Project MACE for MACECalculator with /home/mtr46585/.cache/mace/20231210mace128L0_energy_epoch249model
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
/home/mtr46585/aiida-mlip/.venv/lib/python3.12/site-packages/mace/calculators/mace.py:139: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_l

Finally we can visualise the WorkGraph

In [ ]:
from aiida_workgraph.utils import generate_node_graph

generate_node_graph(wg.pk)

In [ ]:
! verdi process list -a

In [ ]:
! verdi node show 7085  

In [22]:
! python3 sample_split.py --trajectory janus_results/lj-traj-descriptors.extxyz --pre ar --n_samples 10 --config_types=None

create files: train_file=PosixPath('ar-train.xyz'), valid_file=PosixPath('ar-valid.xyz') and test_file=PosixPath('ar-test.xyz')
Processing: ('all', 'lj-traj'), 11 frames
/home/mtr46585/aiida-mlip/.venv/lib/python3.12/site-packages/fpsample/wrapper.py:98: UserWarning: k is too large, set to 11
  warnings.warn(f"k is too large, set to {n_pts}")
  ('all', 'lj-traj'): total=11, train_target=8, vt_target=2
/home/mtr46585/aiida-mlip/.venv/lib/python3.12/site-packages/fpsample/wrapper.py:98: UserWarning: k is too large, set to 3
  warnings.warn(f"k is too large, set to {n_pts}")
Found 0 structures that were too similar during sampling.
